# Air analysis

We have a 1% sample of all air legs flown in Q2 2022. We will use this to derive basic information about air flows in the US.

This data is extracted from the Bureau of Transportation Statistics DB1B dataset.

In [ ]:
import pandas as pd
import numpy as np

First, we need to load our data from the provided CSV file.

In [ ]:
data = pd.read_csv("data/air_sample.csv")

We can look at a sample of the data to see what it looks like.

In [ ]:
data.iloc[0:10]

The data have seven columns: origin and destination airport, origin and destination cities carrier, and distance. The city and carrier are coded, so we will merge in other (the airports have codes as well, but these are fairly well known - e.g. RDU is Raleigh-Durham and LAX is Los Angeles; we won't match those with the official airport names)

In [ ]:
market_ids = pd.read_csv("data/L_CITY_MARKET_ID.csv")

data = data.merge(
    market_ids.rename(columns={"Description": "OriginCity"}).set_index("Code"),
    left_on="OriginCityMarketID",
    right_index=True
)

data = data.merge(
    market_ids.rename(columns={"Description": "DestCity"}).set_index("Code"),
    left_on="DestCityMarketID",
    right_index=True
)

In [ ]:
carriers = pd.read_csv("data/L_CARRIERS.csv")

data = data.merge(
    carriers.rename(columns={"Description": "OperatingCarrierName"}).set_index("Code"),
    left_on="OpCarrier",
    right_index=True
)

data = data.merge(
    carriers.rename(columns={"Description": "TicketingCarrierName"}).set_index("Code"),
    left_on="TkCarrier",
    right_index=True
)

## Most popular air routes

Now, we can see what the most popular air routes are, by summing up the number of passengers carried.

In [ ]:
pairs = (
    data
        .groupby(["Origin", "Dest"], as_index=False)
        .agg({"Passengers": "sum", "Distance": "first"})
        .sort_values("Passengers", ascending=False)
)
pairs

We see that LAX-JFK (Los Angeles to New York Kennedy) is represented separately from JFK-LAX. We'd like to combine these two. Create airport1 and airport2 fields with the first and second airport in alphabetical order.

In [ ]:
data["airport1"] = np.where(data.Origin < data.Dest, data.Origin, data.Dest)
data["airport2"] = np.where(data.Origin < data.Dest, data.Dest, data.Origin)

Now, we repeat the analysis with our new columns.

In [ ]:
pairs = (
    data
        .groupby(["airport1", "airport2"], as_index=False)
        .agg({"Passengers": "sum", "Distance": "first"})
        .sort_values("Passengers", ascending=False)
)
pairs

## Market-level analysis

This may be misleading, however, as some metropolitan areas have only one airport (for example, Raleigh-Durham or Las Vegas), while others have more (for example, New York or Los Angeles). We can repeat the analysis grouping by "market", which groups these airports together. Now, we can see what the most popular air route is, by summing up the number of passengers carried.

In [ ]:
data["city1"] = np.where(data.OriginCity < data.DestCity, data.OriginCity, data.DestCity)
data["city2"] = np.where(data.OriginCity < data.DestCity, data.DestCity, data.OriginCity)

In [ ]:
pairs = (
    data
        .groupby(["city1", "city2"], as_index=False)
        .agg({"Passengers": "sum", "Distance": "first"})
        .sort_values("Passengers", ascending=False)
)

pairs["distance_km"] = pairs.Distance * 1.609

pairs